<a href="https://colab.research.google.com/github/gndumbri/Python-Side-Projects/blob/master/PharmCAS_Web_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get update

In [0]:
!apt install chromium-chromedriver

In [0]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.colab import files
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [0]:
driver = webdriver.Chrome('chromedriver',options=options)
driver.get("http://www.pharmcas.org/school-directory/#/pharmd/general-information")
driver.implicitly_wait(10)

In [0]:
numbers = driver.find_elements(By.XPATH, 
          '//table[@class="phc-schools-table phc-schools-table-pharmd"]//a')
links = []
for n in numbers:
    number = n.get_attribute('href')
    links.append(number)

len(links)

166

In [0]:
len(links[11:])

155

In [0]:
def pharm_crawl(a_list, data_list):
    headers = {'user-agent': 'jobscraper - school project (gndumbri@gmail.com)'}
    for i in a_list:
        url = i
        response = requests.get(url, headers=headers)
        if response.ok:
            data = response.text

            soup = BeautifulSoup(data, 'html.parser')
            
            name = soup.select('h1')
            names = [nom.get_text() for nom in name]
            all_names = ''.join(names[0]).strip()

            if soup.find("span", itemprop="addressRegion") is not None:
                state = soup.find("span", itemprop="addressRegion").get_text()
            else:
                state = 'N/A'

            deadline = soup.select('.deadline > table > tbody > tr > \
            td:nth-of-type(2)')
            deadlines = [dead.get_text() for dead in deadline]
            all_deadlines = ''.join(deadlines)

            seat = soup.select('.prog-statistics > table > tbody > \
            tr:nth-of-type(1) > td:nth-of-type(2)')
            seats = [spot.get_text() for spot in seat]
            all_seats = ''.join(seats)

            private = soup.select('.prog-information > table > tbody > \
            tr:nth-of-type(3) > td:nth-of-type(2)')
            statuses = [status.get_text() for status in private]
            all_statuses = ''.join(statuses).strip()

            early = soup.select('.prog-information > table > tbody > \
            tr:nth-of-type(11) > td:nth-of-type(2)')
            early_dec = [first.get_text() for first in early]
            all_early = ''.join(early_dec)
            
            ea = soup.select('.prog-information > table > tbody > \
            tr:nth-of-type(12) > td:nth-of-type(2) > ul > li')
            eas = [e_a.get_text() for e_a in ea]
            all_eas = ''.join(eas)

            min_ovr = soup.select('.prog-criteria > table > tbody > \
            tr:nth-of-type(1) > td:nth-of-type(2)')
            mins = [ovr.get_text() for ovr in min_ovr]
            all_min_ovr = ''.join(mins)

            min_req = soup.select('.prog-criteria > table > tbody > \
            tr:nth-of-type(2) > td:nth-of-type(2)')
            reqs = [req.get_text() for req in min_req]
            all_min_req = ''.join(reqs)

            hour = soup.select('.prog-prerequisites > table:nth-of-type(1) > \
            tbody > tr:nth-of-type(1) > td:nth-of-type(2)')
            hours = [our.get_text() for our in hour]
            all_hours = ''.join(hours)

            pcat = soup.select('.test > table > tbody > tr:nth-of-type(1) > \
            td:nth-of-type(2) > p')
            scores = [score.get_text() for score in pcat]
            all_scores = ''.join(scores).strip()

            lor = soup.select('.letters-of-reference > table > tbody > \
            tr:nth-of-type(2) > td:nth-of-type(2)')
            lors = [lo.get_text() for lo in lor]
            all_lors = ''.join(lors).strip()

            deposit = soup.select('.accepted-applicants > table > tbody > \
            tr > td > p')
            deposit = [dep.get_text() for dep in deposit]
            deposits = ''.join(deposit).strip()
            
            amount = soup.select('.accepted-applicants > table > tbody > \
            tr > td > table > tbody > tr:nth-of-type(1) > td:nth-of-type(2)')
            amount = [am.get_text() for am in amount]
            amounts = ''.join(amount).strip()

            details = soup.select('.accepted-applicants > table > tbody > \
            tr > td > table > tbody > tr:nth-of-type(2) > td:nth-of-type(2)')
            details = [det.get_text() for det in details]
            details = ''.join(details).replace('\r', '')
            details = ''.join(details).replace('\n', '')

            details_2 = soup.select('.accepted-applicants > table > tbody > \
            tr > td > table > tbody > tr:nth-of-type(3) > td:nth-of-type(2)')
            details_2 = [det.get_text() for det in details_2]
            details_2 = ''.join(details_2).replace('\r', '')
            details_2 = ''.join(details_2).replace('\n', '')

            deposit_info = deposits + ': ' + amounts + ' Refundable: ' \
            + details + ' - ' + details_2

        data_list.append((all_names, state, all_deadlines, 
                          all_seats, all_statuses, 
                          all_early, all_eas, all_min_ovr, all_min_req, 
                          all_hours, all_scores, all_lors, deposit_info))


In [0]:
data_list = []
pharm_crawl(links[11:], data_list)

df = pd.DataFrame(data_list, columns=['School', 
                  'State', 'Application Deadline', 
                  'Class Size','Inst Type', 'Early Decision',
                  'Specialty Programs', 'Min Overall GPA',
                  'Min Pre-Req GPA', 'Pre-req hours',
                  'PCAT Required', 'LORs', 'Deposit'])
df.head()

,School,State,Application Deadline,Class Size,Inst Type,Early Decision,Specialty Programs,Min Overall GPA,Min Pre-Req GPA,Pre-req hours,PCAT Required,LORs,Deposit
0,Albany College of Pharmacy and Health Sciences...,New York,"June 1, 2020",170,Private,Yes,Early Assurance,2.5,2.5,63,Required,Two (2),Yes: $500.00 Refundable: No -
1,Albany College of Pharmacy and Health Sciences...,Vermont,"May 1, 2020",30,Private,Yes,Early Assurance Accelerated Progression (year...,2.5,2.5,63,Required,Two (2),Yes: $500.00 Refundable: No -
2,American University of Health Sciences,California,"June 1, 2020",60,Private,No,None,2.5,2.0,57,Optional,Three (3),Yes: $500 Refundable: No - Applicants accepted...
3,Appalachian College of Pharmacy,Virginia,"July 1, 2020",75,Private,Yes,None,2.00,2.00,72,Optional,Three (3),"Yes: $1000.00 Refundable: Yes - The $1,000 sea..."
4,Auburn University,Alabama,"July 1, 2020",151,Public,Yes,Early Assurance,2.5,2.5,87,Optional,Three (3),Yes: $500 Refundable: No - Applicants admitted...


In [0]:
df.to_csv('Pharmacy_School_Surveillance_Program.csv', index = None, 
          header=True, encoding='utf-8')

In [0]:
files.download('Pharmacy_School_Surveillance_Program.csv')

In [0]:
# testing
my_list = []
url = 'http://schoolpages.pharmcas.org/publishedsurvey/414'
response = requests.get(url)
if response.ok:

    data = response.text

    pcat_score = soup.select('.test > table > tbody > \
    tr:nth-of-type(2) > td:nth-of-type(2)')
    pcat_scores = [pcs.get_text() for pcs in pcat_score]
    all_pcat_scores = ''.join(pcat_scores)



In [0]:
all_pcat_scores

''

In [0]:
def alert_crawl(a_list, data_list):
    for i in a_list:
        url = i
        response = requests.get(url)
        if response.ok:

            data = response.text

            soup = BeautifulSoup(data, 'html.parser')

            name = soup.select('h1')
            names = [nom.get_text() for nom in name]
            school_names = ''.join(names[0]).strip()

            if soup.find("span", itemprop="addressRegion") is not None:
                state = soup.find("span", itemprop="addressRegion").get_text()
            else:
                state = 'N/A'            
            
            alert_1 = soup.select('aside:nth-of-type(1)')
            alert_1 = ''.join([aler.get_text().strip() for aler in alert_1]) 
            if 'Additional Information:' in alert_1:
                alert_1 = 'N/A'
            elif soup.select('aside:nth-of-type(1)') == []:
                alert_1 = 'N/A'

            alert_2 = soup.select('.prog-information > aside')
            if 'Additional Information:' in alert_2:
                alert_2 = 'N/A'
            elif soup.select('.prog-information > aside') == []:
                alert_2 = 'N/A'
            else: 
                alert_2 = ''.join([aler.get_text().strip() for aler in alert_2])  

            alert_3 = soup.select('.prog-criteria > aside')
            if 'Details:' in alert_3:
                alert_3 = 'N/A'
            elif 'Additional Information:' in alert_3:
                alert_3 = 'N/A'
            elif soup.select('.prog-criteria > aside') == []:
                alert_3 = 'N/A'
            else: 
                alert_3 = ''.join([aler.get_text().strip() for aler in alert_3]) 

        data_alerts.append((school_names, state, alert_1, alert_2, alert_3))

In [0]:
data_alerts = []
alert_crawl(links[11:], data_alerts)
alerts = pd.DataFrame(data_alerts, columns=['School Name', 'State', 'Alert 1', 'Alert 2', 'Alert 3'])
alerts.head()

,School Name,State,Alert 1,Alert 2,Alert 3
0,Albany College of Pharmacy and Health Sciences...,New York,ACPHS offers a 4-year pharmacy degree at its A...,N/A,N/A
1,Albany College of Pharmacy and Health Sciences...,Vermont,ACPHS offers a 3-year pharmacy degree at its C...,N/A,N/A
2,American University of Health Sciences,California,N/A,N/A,N/A
3,Appalachian College of Pharmacy,Virginia,Apply now! THE APPLICATION DEADLINE HAS BEEN ...,N/A,N/A
4,Auburn University,Alabama,N/A,N/A,N/A


In [0]:
alerts.to_csv('PharmCAS Alerts.csv', index = None, 
          header=True, encoding='utf-8')

In [0]:
files.download('PharmCAS Alerts.csv')